# Steady State - Aggregate

**Table of contents**<a id='toc0_'></a>    
- 1. [Setup](#toc1_)    
- 2. [Find steady state](#toc2_)    
- 3. [Speed and error tests](#toc3_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [1]:
%load_ext autoreload
%autoreload 2

import time
import pickle
import numpy as np
from scipy import optimize

import matplotlib.pyplot as plt   
plt.rcParams.update({"axes.grid":True,"grid.color":"black","grid.alpha":"0.25","grid.linestyle":"--"})
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

from BabyMAKROModel import BabyMAKROModelClass

## 1. <a id='toc1_'></a>[Setup](#toc0_)

In [2]:
model = BabyMAKROModelClass(name='baseline') # create an instance of the model

## 2. <a id='toc2_'></a>[Find steady state](#toc0_)

In [3]:
par = model.par
model.find_ss(do_print=True)    

Labor supply, search and matching:
ss.S/par.N_work = 0.15, ss.L/par.N_work = 0.96, ss.U/par.N_work = 0.04
ss.delta_L = 0.12, ss.m_s = 0.75, ss.m_v = 0.75, ss.v = 6.93
par.sigma_m = 0.42
Capital agency FOC:
ss.r_K = 0.12
Labor agency FOC:
ss.r_ell = 1.01, (ss.L-ss.ell)/par.N_work*100 = 0.77
Production firm:
ss.P_Y_0 = 0.91, ss.Gamma = 0.54, ss.Y = 71.99, ss.K = 184.37
Capital accumulation:
ss.iota = 18.44, ss.I = 18.44
Government:
ss.B = 0.00, ss.G = 21.60, ss.tau = 0.60
Households:
ss.Aq/par.N = 0.02, ss.real_W = 1.00, ss.C = 21.92, ss.A = 12.44, ss.r_hh = 0.02
Market clearing:
ss.C/ss.Y = 0.30, ss.G/ss.Y = 0.30, ss.I/ss.Y = 0.26, ss.X/ss.Y = 0.58, ss.M/ss.Y = 0.44


## 3. <a id='toc3_'></a>[Speed and error tests](#toc0_)

In [4]:
model.set_exo_ss()          
model.set_unknowns_ss()     

**Python evaluation:**

In [5]:
%time model.evaluate_blocks(py=True)

CPU times: total: 7.25 s
Wall time: 7.81 s


**Check errors:**

In [6]:
errors = model.get_errors(do_print=True)

A_R_ini_error       : abs. max = 1.90e-14
Aq_diff             : abs. max = 3.73e-15
FOC_capital_agency  : abs. max = 2.22e-16
FOC_K_ell           : abs. max = 2.66e-15
mkt_clearing        : abs. max = 0.00e+00
PC                  : abs. max = 0.00e+00


In [7]:
if np.allclose(errors,0.0):
    print('errors are zero')
else:
    raise ValueError('errors are NOT zero')

errors are zero


**First numba evaluation:**

In [8]:
%time model.evaluate_blocks()

CPU times: total: 3min 11s
Wall time: 3min 18s


**Time:**

In [9]:
%timeit model.evaluate_blocks()
%timeit model.get_errors()

137 ms ± 4.56 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
83.5 µs ± 991 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [10]:
%timeit model.evaluate_blocks(py=True)

7.12 s ± 731 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
